In [5]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
#load the data directory
train_dir = "C:\\Users\ss\Desktop\m.sc(dsa)\CV_project\TRAIN"

In [7]:
datagen = ImageDataGenerator(
    rescale=1.0/255,                # Normalize pixel values
    rotation_range=20,             # Rotate images up to 20 degrees
    width_shift_range=0.2,         # Shift the width of the image
    height_shift_range=0.2,        # Shift the height of the image
    shear_range=0.2,               # Shear transformations
    zoom_range=0.2,                # Zoom in/out
    horizontal_flip=True,          # Flip horizontally
    vertical_flip=True,            # Flip vertically
    fill_mode='nearest'            # Fill in missing parts
)

In [13]:

# Load and preprocess the data
batch_size = 32  
image_size = (224, 224) 
class_mode = 'categorical'


train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode=class_mode,
    shuffle=True,  # Shuffle the data
    subset='training'  # For training data
)

Found 2106 images belonging to 2 classes.


In [10]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

# Load the pre-trained ResNet-50 model
base_model = ResNet50(weights='imagenet', include_top=False)

# Add custom layers for your classification task
num_classes = 2  #class used for classification task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [11]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define early stopping and model checkpoint callbacks
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)
num_epochs = 50  # Adjust the number of training epochs as needed

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    callbacks=[early_stopping, model_checkpoint]
)


Epoch 1/50
66/66 [==============================] - 1046s 15s/step - loss: 0.6566 - accuracy: 0.7840
Epoch 2/50
66/66 [==============================] - 1004s 15s/step - loss: 0.3550 - accuracy: 0.8571
Epoch 3/50
66/66 [==============================] - 1002s 15s/step - loss: 0.3155 - accuracy: 0.8694
Epoch 4/50
66/66 [==============================] - 1002s 15s/step - loss: 0.3210 - accuracy: 0.8699
Epoch 5/50
66/66 [==============================] - 1001s 15s/step - loss: 0.3000 - accuracy: 0.8775
Epoch 6/50
66/66 [==============================] - 1003s 15s/step - loss: 0.2996 - accuracy: 0.8794
Epoch 7/50
66/66 [==============================] - 1001s 15s/step - loss: 0.2497 - accuracy: 0.9017
Epoch 8/50
66/66 [==============================] - 1006s 15s/step - loss: 0.2557 - accuracy: 0.8922
Epoch 9/50
66/66 [==============================] - 1002s 15s/step - loss: 0.2530 - accuracy: 0.8984
Epoch 10/50
66/66 [==============================] - 1002s 15s/step - loss: 0.2286 - accura

In [27]:
#saving and loading the model trained

from tensorflow.keras.models import load_model, save_model

model.save('best_model.h5')

best_model = load_model('best_model.h5')

In [28]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Load your test data or validation data 
test_data_dir = "C:\\Users\ss\Desktop\m.sc(dsa)\CV_project\TEST"
test_generator = datagen.flow_from_directory(
    test_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Do not shuffle for validation/test data
)

# Make predictions on the test data
predictions = best_model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)  # Convert to class labels

# True labels for the test data (ground truth)
true_labels = test_generator.classes

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate precision, recall, and F1-score for each class
precision = precision_score(true_labels, predicted_labels, average=None)
recall = recall_score(true_labels, predicted_labels, average=None)
f1 = f1_score(true_labels, predicted_labels, average=None)

# Classification report (includes precision, recall, F1-score, and support for each class)
class_names = list(test_generator.class_indices.keys())
class_report = classification_report(true_labels, predicted_labels, target_names=class_names)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
print("Classification Report:\n", class_report)


Found 374 images belonging to 2 classes.
12/12 [==============================] - 53s 4s/step
Accuracy: 0.9171122994652406
Precision: [0.8872549  0.95294118]
Recall: [0.95767196 0.87567568]
F1-Score: [0.92111959 0.91267606]
Classification Report:
               precision    recall  f1-score   support

       COVID       0.89      0.96      0.92       189
   non-COVID       0.95      0.88      0.91       185

    accuracy                           0.92       374
   macro avg       0.92      0.92      0.92       374
weighted avg       0.92      0.92      0.92       374

